# 勉強： ジェネレータをイテレータから理解するを読んだ．

[Python: ジェネレータをイテレータから理解する](https://blog.amedama.jp/entry/2017/11/23/213233)をよんで，ジェネレータとイテレータについて勉強した．
ほとんどトレースみたいになっちゃうけど，ところどころメモとか入れていく．
## イテレータとは
イテレータについて，いくつかの側面から見た実態を例にとって解説されていた．  
個人的に，忘れていたのがリストとかの ==コンテナオブジェクトはイテレータではない== ．という点．
あと，使い方もよくわかっていなかったのでメモしてく．

In [4]:
list = [1, 2, 3]

listを作って，イテレータオブジェクトをコンテナオブジェクトから取り出す．  
イテレータは組み込み関数 `iter()` を使う．

In [20]:
iterator_object = iter(list)

ここで重要なのが， __コンテナオブジェクトはイテレータではない__ ということ．一般論では，コンテナ型とは複数のオブジェクトの集まりを表現するデータ構造の総称である．その中に，特にPythonでは，シーケンス型の`list, tuple, str`がある．文字列はイミュータブルな文字シーケンス．また，集合型の`set, frozenset`，そして，辞書型の`dict`である．集合型は要素の重複と順序がないデータ構造で，`frozenset`はイミュータブルなもの．辞書型はキーとオブジェクトを持つデータ構造．

もっと厳密にいうとPythonでは,イテレータが使えるオブジェクトをイテレータブルなオブジェクトと言って，`__iter__`メソッドが定義されているオブジェクトで，イテレータは`__iter__`と`__next__`オブジェクトが定義されているオブジェクトのことをいう．コンテナオブジェクトは，`__contains__`を定義されたオブジェクトである．だけど，常識的に，先の一般論で述べたけどコンテナ型は総じてイテレータブルであると言っていい．

だから，実は，参照した記事の記載はちょっとだけニュアンスが違っていて，コンテナオブジェクトに`__iter__`を実装したものみたいに読めるけど，厳密にはそうではない．

イテレータからは，取り出した元のlistオブジェクトの要素オブジェクトを参照する．  
実際に取り出すときには`next()`を使う．

In [21]:
next(iterator_object)

1

In [22]:
next(iterator_object)

2

In [23]:
next(iterator_object)

3

In [24]:
next(iterator_object)

StopIteration: 

全部の要素を参照したあとに`next()`を実行するとStopIteration例外になる．この段階でlistの要素にはアクセスできない．  
イテレータオブジェクトは __使い捨て__ であることに注意．さらに，イテレータはあくまで参照なのでlistのコピーではないことに注意．

for文とかは，実は暗黙的に，イミュータブルな型からiter()を実行してイテレータでnext()参照している．

In [30]:
hoge = ([1], 3, [4])
hoge_itr = iter(hoge)
next(hoge_itr)

[1]

In [33]:
hoge[2].append(5)
hoge

([1], 3, [4, 5, 5])

In [34]:
next(hoge_itr)

3

In [35]:
next(hoge_itr)

[4, 5, 5]

イテレータの最大の利点はインタフェースの統一．いくつものコンテナ型がforとかで列挙できること．  
メモリの節約に関してはちょっとよくわからなかった．イテレータの定義から見れば，元のコンテナオブジェクトを参照するわけだから，メモリの節約には貢献できないはず．

## ジェネレータとは
イテレータを簡単に実装するための手段．  
とりあえず，下のiterableなクラスの実装を見てみる．コードは[こちらのQiitaの解説記事から](https://qiita.com/knknkn1162/items/17f7f370a2cc27f812ee#container)引用している．何を言いたいかというと，とにかく面倒くさい．ということ．そこで出てくるのがジェネレータ．

In [37]:
import collections.abc
### 引用：https://qiita.com/knknkn1162/items/17f7f370a2cc27f812ee#container

# iterableであるために
# __iter__(iteration/iterator protocol : 反復プロトコルというらしい)を実装する
class Foo(collections.abc.Iterable):
    def __init__(self):
        self._L = [0,1,2]

    def __iter__(self):
        return FooIterator(self)

#  __next__を実装する必要がある
## (collections.abc.Iteratorを継承しているので、__iter__は自然と定義される)
class FooIterator(collections.abc.Iterator):
    def __init__(self, foo): # fooはiterableなオブジェクト
        self._i = 0
        self._foo = foo

    def __next__(self):
        try:
            v = self._foo._L[self._i]
            self._i += 1
            return v
        # StopIteration(RuntimeError)を送出する
        except IndexError:
            raise StopIteration

とにかく，わざわざイテレータを使いたいがためにこんな事をしないといけないのは正直しんどい．そんな時にジェネレータという特殊な関数を用いる．
実際に，上記のコードをジェネレータで書き直す．

In [42]:
def foo():
    L = [0,1,2]
    i = 0
    while True:
        try:
            yield L[i]
            i+=1
        except IndexError:
            raise StopIteration
foo_itr = foo()
next(foo_itr)

0

In [43]:
next(foo_itr)

1

In [44]:
next(foo_itr)

2

In [45]:
next(foo_itr)

/home/tiwasaki/.pyenv/versions/3.6.5/envs/opencv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: generator 'foo' raised StopIteration
  """Entry point for launching an IPython kernel.


StopIteration: 

ということで，簡単になった．じっさいにfoo()はどんなオブジェクトかみてみたい．

In [46]:
type(foo_itr)

generator

これはジェネレータイテレータオブジェクトというらしい．面倒だからジェネレータでいいと思う．  
重要なのは，`yield`がイテレータの`__next__`特殊関数に対応している．という点．イテレータではないけど，まるでイテレータのように振る舞う．オブジェクトであることがわかった．

### ジェネレータの最大の利点
これこそメモリの節約だと思われる．例えば，数列など無限にあるものはジェネレータが最適．更に巨大なファイル読み込みなども注目したい．

## 参考にしたサイト
[1](https://blog.amedama.jp/entry/2017/11/23/213233)
[2](https://qiita.com/knknkn1162/items/17f7f370a2cc27f812ee#container)